In [1]:
import pandas as pd
import numpy as np 
from tabulate import tabulate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import metrics
nltk.download('vader_lexicon')
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report
from sklearn.decomposition import LatentDirichletAllocation

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/prashantkarna/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
import pandas as pd
data1 = pd.read_csv('/Users/prashantkarna/Documents/Research Materials/Datasets/Suicide_Detection.csv')
data1['class'] = data1['class'].replace(['suicide'], 1)
data1['class'] = data1['class'].replace(['non-suicide'], 0)



In [3]:
import pandas as pd
data2 = pd.read_csv('/Users/prashantkarna/Documents/Research Materials/Datasets/depression_dataset_reddit_cleaned.csv')
# rename the columns
data2 = data2.rename(columns={'clean_text': 'text', 'is_depression': 'class'})

In [4]:
data = pd.concat([data1,data2])
data.shape

(239805, 3)

In [5]:
# Count the number of instances in each class
class_counts = data['class'].value_counts()

# Print the class counts
print(class_counts)

0    119937
1    119868
Name: class, dtype: int64


In [6]:
import numpy as np
data_split = np.array_split(data, 3)
data = data_split[0]
data = data.drop('Unnamed: 0',axis=1)

In [7]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/prashantkarna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
X = data.drop('class', axis=1)
y = data['class']


In [9]:
# To remove emails
email_regex = r'([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
regexes_to_remove = [email_regex, r'Subject:', r'Re:']

for i in range(0, len(X)):
    # removing all special charachter
    review = re.sub('[^a-zA-Z]', ' ', str(X['text'][i]))
    # make document as lowerCase
    review = review.lower()
    # splitting the documents into words for ex ['iam', 'omar']
    review = review.split()
    # make limmatization --> (change, changing, changes)---> (change)
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    # join the document agian
    review = ' '.join(review)
    
    # removing mails
    for r in regexes_to_remove:
        X['text'][i] = re.sub(r, '', review)


In [10]:
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
tfidf_vectorizer_uni = TfidfVectorizer(max_features=10000,ngram_range=(1,1))
tfidf_vectorizer_bi = TfidfVectorizer(max_features=10000, ngram_range=(1,2))
tfidf_vectorizer_tri = TfidfVectorizer(max_features=10000, ngram_range=(1,3))

X_tfidf = tfidf_vectorizer.fit_transform(X['text'])
# X_tfidf_test = tfidf_vectorizer.transform(X['text'])

X_tfidf_uni = tfidf_vectorizer_uni.fit_transform(X['text'])
# X_tfidf_test_uni = tfidf_vectorizer_uni.transform(X['text'])

X_tfidf_bi = tfidf_vectorizer_bi.fit_transform(X['text'])
# X_tfidf_test_bi =tfidf_vectorizer_bi.transform(X'text'])

X_tfidf_tri = tfidf_vectorizer_tri.fit_transform(X['text'])
# X_tfidf_test_tri = tfidf_vectorizer_tri.transform(X_test['text'])

In [12]:
# LDA feature extraction on unigrams
lda_uni = LatentDirichletAllocation(n_components=3)
lda_uni.fit(X_tfidf_uni)
lda_features_uni = lda_uni.transform(X_tfidf_uni)
data_lda_uni = pd.DataFrame(lda_features_uni, columns=['topic_1', 'topic_2', 'topic_3'])

# LDA feature extraction on bigrams
lda_bi = LatentDirichletAllocation(n_components=3)
lda_bi.fit(X_tfidf_bi)
lda_features_bi = lda_bi.transform(X_tfidf_bi)
data_lda_bi = pd.DataFrame(lda_features_bi, columns=['topic_1', 'topic_2', 'topic_3'])

# LDA feature extraction on trigrams
lda_tri = LatentDirichletAllocation(n_components=3)
lda_tri.fit(X_tfidf_tri)
lda_features_tri = lda_tri.transform(X_tfidf_tri)
data_lda_tri = pd.DataFrame(lda_features_tri, columns=['topic_1', 'topic_2', 'topic_3'])

In [ ]:
# # LDA feature extraction on unigrams
# lda_uni = LatentDirichletAllocation(n_components=3)
# lda_uni.fit(X_tfidf_train_uni)
# lda_features_uni = lda_uni.transform(X_tfidf_train_uni)
# data_lda_uni = pd.DataFrame(lda_features_uni, columns=['topic_1', 'topic_2', 'topic_3'])

# # LDA feature extraction on bigrams
# lda_bi = LatentDirichletAllocation(n_components=3)
# lda_bi.fit(X_tfidf_train_bi)
# lda_features_bi = lda_bi.transform(X_tfidf_train_bi)
# data_lda_bi = pd.DataFrame(lda_features_bi, columns=['topic_1', 'topic_2', 'topic_3'])

# # LDA feature extraction on trigrams
# lda_tri = LatentDirichletAllocation(n_components=3)
# lda_tri.fit(X_tfidf_train_tri)
# lda_features_tri = lda_tri.transform(X_tfidf_train_tri)
# data_lda_tri = pd.DataFrame(lda_features_tri, columns=['topic_1', 'topic_2', 'topic_3'])

In [13]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

# Feature selection using ExtraTreesClassifier and SelectFromModel

selector = SelectFromModel(ExtraTreesClassifier(n_estimators=300, random_state=42))

                                        
# Unigram feature selection
selector_tfidf= selector.fit(X_tfidf, y)
X_selection = selector_tfidf.transform(X_tfidf)


# Unigram feature selection
selector_uni= selector.fit(X_tfidf_uni, y)
X_selection_uni = selector_uni.transform(X_tfidf_uni)

# Bigram feature selection
selector_bi = selector.fit(X_tfidf_bi, y)
X_selection_bi = selector_bi.transform(X_tfidf_bi)

# Trigram feature selection
selector_tri= selector.fit(X_tfidf_tri, y)
X_selection_tri = selector_tri.transform(X_tfidf_tri)

In [14]:
# Perform feature selection using LDA and ExtraTreesClassifier on unigrams
selector_lda_uni = SelectFromModel(ExtraTreesClassifier(n_estimators=300, random_state=42))
X_selection_lda_uni = selector_lda_uni.fit_transform(lda_features_uni, y)

# Perform feature selection using LDA and ExtraTreesClassifier on bigrams
selector_lda_bi = SelectFromModel(ExtraTreesClassifier(n_estimators=300, random_state=42))
X_selection_lda_bi = selector_lda_bi.fit_transform(lda_features_bi, y)

# Perform feature selection using LDA and ExtraTreesClassifier on trigrams
selector_lda_tri = SelectFromModel(ExtraTreesClassifier(n_estimators=300, random_state=42))
X_selection_lda_tri = selector_lda_tri.fit_transform(lda_features_tri, y)

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Split data into training and testing sets for unigrams
X_train, X_test, y_train, y_test = train_test_split(X_selection_lda_uni, y, test_size=0.2, random_state=42)

# Random Forest Classifier on LDA features for unigrams
rf_uni = RandomForestClassifier(random_state=42)
rf_uni.fit(X_train, y_train)
y_pred_rf_uni = rf_uni.predict(X_test)
print("Random Forest Classifier (LDA features for unigrams):\n", classification_report(y_test, y_pred_rf_uni))

# SVM Classifier on LDA features for unigrams
svm_uni = SVC(random_state=42)
svm_uni.fit(X_train, y_train)
y_pred_svm_uni = svm_uni.predict(X_test)
print("SVM Classifier (LDA features for unigrams):\n", classification_report(y_test, y_pred_svm_uni))

# Logistic Regression on LDA features for unigrams
lr_uni = LogisticRegression(random_state=42)
lr_uni.fit(X_train, y_train)
y_pred_lr_uni = lr_uni.predict(X_test)
print("Logistic Regression (LDA features for unigrams):\n", classification_report(y_test, y_pred_lr_uni))

Random Forest Classifier (LDA features for unigrams):
               precision    recall  f1-score   support

           0       0.87      0.86      0.86      8035
           1       0.86      0.87      0.86      7952

    accuracy                           0.86     15987
   macro avg       0.86      0.86      0.86     15987
weighted avg       0.86      0.86      0.86     15987

SVM Classifier (LDA features for unigrams):
               precision    recall  f1-score   support

           0       0.90      0.85      0.87      8035
           1       0.86      0.90      0.88      7952

    accuracy                           0.88     15987
   macro avg       0.88      0.88      0.88     15987
weighted avg       0.88      0.88      0.88     15987

Logistic Regression (LDA features for unigrams):
               precision    recall  f1-score   support

           0       0.91      0.84      0.87      8035
           1       0.85      0.91      0.88      7952

    accuracy                    

In [16]:
# XGBoost Classifier on LDA features for unigram
xgb_uni = XGBClassifier(random_state=42)
xgb_uni.fit(X_train, y_train)
y_pred_xgb_uni = xgb_uni.predict(X_test)
print("XGBoost Classifier (LDA features for bigrams):\n", classification_report(y_test, y_pred_xgb_uni))

XGBoost Classifier (LDA features for bigrams):
               precision    recall  f1-score   support

           0       0.90      0.86      0.88      8035
           1       0.86      0.90      0.88      7952

    accuracy                           0.88     15987
   macro avg       0.88      0.88      0.88     15987
weighted avg       0.88      0.88      0.88     15987



In [17]:
# Split data into training and testing sets for bigrams
X_train, X_test, y_train, y_test = train_test_split(X_selection_lda_bi, y, test_size=0.2, random_state=42)

# Random Forest Classifier on LDA features for bigrams
rf_bi = RandomForestClassifier(random_state=42)
rf_bi.fit(X_train, y_train)
y_pred_rf_bi = rf_bi.predict(X_test)
print("Random Forest Classifier (LDA features for bigrams):\n", classification_report(y_test, y_pred_rf_bi))

# SVM Classifier on LDA features for bigrams
svm_bi = SVC(random_state=42)
svm_bi.fit(X_train, y_train)
y_pred_svm_bi = svm_bi.predict(X_test)
print("SVM Classifier (LDA features for bigrams):\n", classification_report(y_test, y_pred_svm_bi))

# Logistic Regression on LDA features for bigrams
lr_bi = LogisticRegression(random_state=42)
lr_bi.fit(X_train, y_train)
y_pred_lr_bi = lr_bi.predict(X_test)
print("Logistic Regression (LDA features for bigrams):\n", classification_report(y_test, y_pred_lr_bi))

Random Forest Classifier (LDA features for bigrams):
               precision    recall  f1-score   support

           0       0.88      0.88      0.88      8035
           1       0.88      0.88      0.88      7952

    accuracy                           0.88     15987
   macro avg       0.88      0.88      0.88     15987
weighted avg       0.88      0.88      0.88     15987

SVM Classifier (LDA features for bigrams):
               precision    recall  f1-score   support

           0       0.90      0.86      0.88      8035
           1       0.87      0.91      0.89      7952

    accuracy                           0.89     15987
   macro avg       0.89      0.89      0.89     15987
weighted avg       0.89      0.89      0.89     15987

Logistic Regression (LDA features for bigrams):
               precision    recall  f1-score   support

           0       0.90      0.86      0.88      8035
           1       0.87      0.91      0.89      7952

    accuracy                       

In [18]:
# XGBoost Classifier on LDA features for bigrams
xgb_bi = XGBClassifier(random_state=42)
xgb_bi.fit(X_train, y_train)
y_pred_xgb_bi = xgb_bi.predict(X_test)
print("XGBoost Classifier (LDA features for bigrams):\n", classification_report(y_test, y_pred_xgb_bi))

XGBoost Classifier (LDA features for bigrams):
               precision    recall  f1-score   support

           0       0.90      0.89      0.89      8035
           1       0.89      0.90      0.89      7952

    accuracy                           0.89     15987
   macro avg       0.89      0.89      0.89     15987
weighted avg       0.89      0.89      0.89     15987



In [19]:
# Split data into training and testing sets for trigrams
X_train, X_test, y_train, y_test = train_test_split(X_selection_lda_tri , y, test_size=0.2, random_state=42)

# Random Forest Classifier on LDA features for trigrams
rf_tri = RandomForestClassifier(random_state=42)
rf_tri.fit(X_train, y_train)
y_pred_rf_tri = rf_tri.predict(X_test)
print("Random Forest Classifier (LDA features for trigrams):\n", classification_report(y_test, y_pred_rf_tri))

# SVM Classifier on LDA features for trigrams
svm_tri = SVC(random_state=42)
svm_tri.fit(X_train, y_train)
y_pred_svm_tri = svm_tri.predict(X_test)
print("SVM Classifier (LDA features for trigrams):\n", classification_report(y_test, y_pred_svm_tri))

# Logistic Regression on LDA features for trigrams
lr_tri = LogisticRegression(random_state=42)
lr_tri.fit(X_train, y_train)
y_pred_lr_tri = lr_tri.predict(X_test)
print("Logistic Regression (LDA features for trigrams):\n", classification_report(y_test, y_pred_lr_tri))

Random Forest Classifier (LDA features for trigrams):
               precision    recall  f1-score   support

           0       0.80      0.80      0.80      8035
           1       0.80      0.80      0.80      7952

    accuracy                           0.80     15987
   macro avg       0.80      0.80      0.80     15987
weighted avg       0.80      0.80      0.80     15987

SVM Classifier (LDA features for trigrams):
               precision    recall  f1-score   support

           0       0.88      0.83      0.85      8035
           1       0.84      0.89      0.86      7952

    accuracy                           0.86     15987
   macro avg       0.86      0.86      0.86     15987
weighted avg       0.86      0.86      0.86     15987

Logistic Regression (LDA features for trigrams):
               precision    recall  f1-score   support

           0       0.91      0.78      0.84      8035
           1       0.80      0.92      0.86      7952

    accuracy                    

In [20]:
# XGBoost Classifier on LDA features for trigrams
xgb_tri = XGBClassifier(random_state=42)
xgb_tri.fit(X_train, y_train,eval_metric='error')
y_pred_xgb_tri = xgb_tri.predict(X_test)
print("XGBoost Classifier (LDA features for trigrams):\n", classification_report(y_test, y_pred_xgb_tri))

XGBoost Classifier (LDA features for trigrams):
               precision    recall  f1-score   support

           0       0.87      0.84      0.85      8035
           1       0.84      0.88      0.86      7952

    accuracy                           0.85     15987
   macro avg       0.86      0.86      0.85     15987
weighted avg       0.86      0.85      0.85     15987



In [121]:
# print(X_selection)

In [19]:
# # extracted features to csv
# X_df = pd.DataFrame(X_selection.todense())
# X_df.to_csv('/Users/prashantkarna/Documents/Research Materials/final code/feature_dataset.csv', index=False)

In [21]:
# # extracted features to cs
# X_df_uni = pd.DataFrame(X_selection.todense())
# X_df_uni.to_csv('/Users/prashantkarna/Documents/Research Materials/final code/feature_uni_dataset.csv', index=False)

In [22]:
# # extracted features to csv
# X_df_bi = pd.DataFrame(X_selection.todense())
# X_df_bi.to_csv('/Users/prashantkarna/Documents/Research Materials/final code/feature_bi_dataset.csv', index=False)

In [23]:
# # extracted features to csv
# X_df_tri = pd.DataFrame(X_selection.todense())
# X_df_tri.to_csv('/Users/prashantkarna/Documents/Research Materials/final code/feature_tri_dataset.csv', index=False)

In [25]:
# data1 = pd.read_csv('/Users/prashantkarna/Documents/Research Materials/final code/feature_tri_dataset.csv')
# data1.columns

# # X = data1.drop('class', axis=1)
# # y = data1['class']

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '920', '921', '922', '923', '924', '925', '926', '927', '928', '929'],
      dtype='object', length=930)

In [122]:
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

# Set up Stratified KFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define classifiers
svm = LinearSVC()
lr = LogisticRegression(C=500, penalty='l1', solver='saga')
nb = MultinomialNB(alpha=0.01)

# Define lists to store accuracy scores
svm_acc = []
lr_acc = []
nb_acc = []

# Perform stratified k-fold cross-validation on selected features
for train_index, test_index in skf.split(X_selection_bi, y):
    # Split data into training and testing sets
    X_train, X_test = X_selection_bi[train_index], X_selection_bi[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit and predict using LinearSVC
    svm.fit(X_train, y_train)
    svm_pred = svm.predict(X_test)
    svm_acc.append(accuracy_score(y_test, svm_pred))
    print("LinearSVC classification report:\n", classification_report(y_test, svm_pred))
    
    # Fit and predict using Logistic Regression
    lr.fit(X_train, y_train)
    lr_pred = lr.predict(X_test)
    lr_acc.append(accuracy_score(y_test, lr_pred))
    print("Logistic Regression classification report:\n", classification_report(y_test, lr_pred))
    
    # Fit and predict using Naive Bayes
    nb.fit(X_train, y_train)
    nb_pred = nb.predict(X_test)
    nb_acc.append(accuracy_score(y_test, nb_pred))
    print("Naive Bayes classification report:\n", classification_report(y_test, nb_pred))

# # Print best accuracy score for each classifier
# print("LinearSVC best accuracy:", max(svm_acc))
# print("Logistic Regression best accuracy:", max(lr_acc))
# print("Naive Bayes best accuracy:", max(nb_acc))
# # print("XGBoost best accuracy:", max(xgb_acc))



LinearSVC classification report:
               precision    recall  f1-score   support

           0       0.92      0.93      0.93      7999
           1       0.93      0.92      0.92      7988

    accuracy                           0.93     15987
   macro avg       0.93      0.93      0.93     15987
weighted avg       0.93      0.93      0.93     15987

Logistic Regression classification report:
               precision    recall  f1-score   support

           0       0.92      0.93      0.93      7999
           1       0.93      0.92      0.92      7988

    accuracy                           0.93     15987
   macro avg       0.93      0.93      0.93     15987
weighted avg       0.93      0.93      0.93     15987

Naive Bayes classification report:
               precision    recall  f1-score   support

           0       0.94      0.83      0.88      7999
           1       0.85      0.94      0.89      7988

    accuracy                           0.89     15987
   macro avg  

In [123]:
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Set up Stratified KFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define classifiers
svm = LinearSVC(C=1.0,dual=False, max_iter=1000,random_state=42)
lr = LogisticRegression()
nb = MultinomialNB()

# Define lists to store accuracy scores
svm_acc = []
lr_acc = []
nb_acc = []

# Perform stratified k-fold cross-validation on selected features
for train_index, test_index in skf.split(X_selection_tri, y):
    # Split data into training and testing sets
    X_train, X_test = X_selection_tri[train_index], X_selection_tri[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit and predict using LinearSVC
    svm.fit(X_train, y_train)
    svm_pred = svm.predict(X_test)
    svm_acc.append(accuracy_score(y_test, svm_pred))
    print("LinearSVC classification report:\n", classification_report(y_test, svm_pred))
    
    # Fit and predict using Logistic Regression
    lr.fit(X_train, y_train)
    lr_pred = lr.predict(X_test)
    lr_acc.append(accuracy_score(y_test, lr_pred))
    print("Logistic Regression classification report:\n", classification_report(y_test, lr_pred))
    
    # Fit and predict using Naive Bayes
    nb.fit(X_train, y_train)
    nb_pred = nb.predict(X_test)
    nb_acc.append(accuracy_score(y_test, nb_pred))
    print("Naive Bayes classification report:\n", classification_report(y_test, nb_pred))

# Print best accuracy score for each classifier
print("LinearSVC best accuracy:", max(svm_acc))
print("Logistic Regression best accuracy:", max(lr_acc))
print("Naive Bayes best accuracy:", max(nb_acc))



LinearSVC classification report:
               precision    recall  f1-score   support

           0       0.92      0.93      0.92      7999
           1       0.93      0.92      0.92      7988

    accuracy                           0.92     15987
   macro avg       0.92      0.92      0.92     15987
weighted avg       0.92      0.92      0.92     15987

Logistic Regression classification report:
               precision    recall  f1-score   support

           0       0.92      0.93      0.92      7999
           1       0.93      0.91      0.92      7988

    accuracy                           0.92     15987
   macro avg       0.92      0.92      0.92     15987
weighted avg       0.92      0.92      0.92     15987

Naive Bayes classification report:
               precision    recall  f1-score   support

           0       0.93      0.83      0.88      7999
           1       0.85      0.94      0.89      7988

    accuracy                           0.89     15987
   macro avg  

In [124]:
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Set up Stratified KFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define classifiers
svm = LinearSVC()
lr = LogisticRegression()
nb = MultinomialNB()

# Define lists to store accuracy scores
svm_acc = []
lr_acc = []
nb_acc = []

# Perform stratified k-fold cross-validation on selected features
for train_index, test_index in skf.split(X_selection_uni, y):
    # Split data into training and testing sets
    X_train, X_test = X_selection_uni[train_index], X_selection_uni[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit and predict using LinearSVC
    svm.fit(X_train, y_train)
    svm_pred = svm.predict(X_test)
    svm_acc.append(accuracy_score(y_test, svm_pred))
    print("LinearSVC classification report:\n", classification_report(y_test, svm_pred))
    
    # Fit and predict using Logistic Regression
    lr.fit(X_train, y_train)
    lr_pred = lr.predict(X_test)
    lr_acc.append(accuracy_score(y_test, lr_pred))
    print("Logistic Regression classification report:\n", classification_report(y_test, lr_pred))
    
    # Fit and predict using Naive Bayes
    nb.fit(X_train, y_train)
    nb_pred = nb.predict(X_test)
    nb_acc.append(accuracy_score(y_test, nb_pred))
    print("Naive Bayes classification report:\n", classification_report(y_test, nb_pred))

# Print best accuracy score for each classifier
print("LinearSVC best accuracy:", max(svm_acc))
print("Logistic Regression best accuracy:", max(lr_acc))
print("Naive Bayes best accuracy:", max(nb_acc))


LinearSVC classification report:
               precision    recall  f1-score   support

           0       0.92      0.93      0.93      7999
           1       0.93      0.92      0.92      7988

    accuracy                           0.92     15987
   macro avg       0.93      0.92      0.92     15987
weighted avg       0.93      0.92      0.92     15987

Logistic Regression classification report:
               precision    recall  f1-score   support

           0       0.91      0.93      0.92      7999
           1       0.93      0.91      0.92      7988

    accuracy                           0.92     15987
   macro avg       0.92      0.92      0.92     15987
weighted avg       0.92      0.92      0.92     15987

Naive Bayes classification report:
               precision    recall  f1-score   support

           0       0.93      0.83      0.88      7999
           1       0.84      0.94      0.89      7988

    accuracy                           0.88     15987
   macro avg  

In [125]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Set up Stratified KFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Define list to store accuracy scores
rf_acc = []

# Perform stratified k-fold cross-validation on selected features
for train_index, test_index in skf.split(X_selection_tri, y):
    # Split data into training and testing sets
    X_train, X_test = X_selection_tri[train_index], X_selection_tri[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit and predict using Random Forest Classifier
    rf.fit(X_train, y_train)
    rf_pred = rf.predict(X_test)
    rf_acc.append(accuracy_score(y_test, rf_pred))

# Print best accuracy score for Random Forest Classifier
print("Random Forest Classifier best accuracy:", max(rf_acc))


Random Forest Classifier best accuracy: 0.899793582285607


In [127]:
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Set up Stratified KFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define XGBoost Classifier
xgb = XGBClassifier(random_state=42)

# Define list to store accuracy scores
xgb_acc = []

# Perform stratified k-fold cross-validation on selected features
for train_index, test_index in skf.split(X_selection_tri, y):
    # Split data into training and testing sets
    X_train, X_test = X_selection_tri[train_index], X_selection_tri[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit and predict using XGBoost Classifier
    xgb.fit(X_train, y_train, eval_metric='error')
    xgb_pred = xgb.predict(X_test)
    xgb_acc.append(accuracy_score(y_test, xgb_pred))

# Print best accuracy score for XGBoost Classifier
print("XGBoost Classifier tri best accuracy:", max(xgb_acc))

#  Perform stratified k-fold cross-validation on selected features
for train_index, test_index in skf.split(X_selection_bi, y):
    # Split data into training and testing sets
    X_train, X_test = X_selection_bi[train_index], X_selection_bi[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit and predict using XGBoost Classifier
    xgb.fit(X_train, y_train, eval_metric='error')
    xgb_pred = xgb.predict(X_test)
    xgb_acc.append(accuracy_score(y_test, xgb_pred))

# Print best accuracy score for XGBoost Classifier
print("XGBoost Classifier bi  best accuracy:", max(xgb_acc))

# Print best accuracy score for XGBoost Classifier
print("XGBoost Classifier tri best accuracy:", max(xgb_acc))

#  Perform stratified k-fold cross-validation on selected features
for train_index, test_index in skf.split(X_selection_uni, y):
    # Split data into training and testing sets
    X_train, X_test = X_selection_uni[train_index], X_selection_uni[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit and predict using XGBoost Classifier
    xgb.fit(X_train, y_train, eval_metric='error')
    xgb_pred = xgb.predict(X_test)
    xgb_acc.append(accuracy_score(y_test, xgb_pred))

# Print best accuracy score for XGBoost Classifier
print("XGBoost Classifier uni  best accuracy:", max(xgb_acc))


XGBoost Classifier tri best accuracy: 0.9139926190029399
XGBoost Classifier bi  best accuracy: 0.9160567961468693
XGBoost Classifier tri best accuracy: 0.9160567961468693
XGBoost Classifier uni  best accuracy: 0.9160567961468693


AdaBoost Classifier best accuracy: 0.8819080860965678


Accuracy: 0.9273479413095469
